We start by downloading the necessary libraries.

In [0]:
!git clone https://github.com/adines/unet.git

Cloning into 'unet'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 398 (delta 0), reused 4 (delta 0), pack-reused 392
Receiving objects: 100% (398/398), 30.92 MiB | 29.26 MiB/s, done.
Resolving deltas: 100% (140/140), done.


Download the images.

In [0]:
!wget https://www.dropbox.com/s/30j6fugdwekgypd/unetData.zip?dl=0 -O unetData.zip
!unzip unetData.zip

--2019-01-28 16:08:43--  https://www.dropbox.com/s/30j6fugdwekgypd/unetData.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/30j6fugdwekgypd/unetData.zip [following]
--2019-01-28 16:08:43--  https://www.dropbox.com/s/raw/30j6fugdwekgypd/unetData.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc30e9b63cdfd9dfd2d397fb863d.dl.dropboxusercontent.com/cd/0/inline/AaQJ6_6HgSZ7jU-jekC_mtE0jG05m7l1qE3JHpa9g0oaswFZa3O8GvrF0S7tZNGbMYdnRfSbhZqiaQ5mnk8W2xmmKAhTHCCTWvagseIqJ-1Gsg/file# [following]
--2019-01-28 16:08:44--  https://uc30e9b63cdfd9dfd2d397fb863d.dl.dropboxusercontent.com/cd/0/inline/AaQJ6_6HgSZ7jU-jekC_mtE0jG05m7l1qE3JHpa9g0oaswFZa3O8GvrF0S7tZNGbMYdnRfSbhZqiaQ5mnk8W2xmmKAhTHCCTWvagseIqJ-1Gsg/file
Resolving uc30

Install some modules.

In [0]:
!pip install clodsa

    100% |████████████████████████████████| 1.5MB 18.3MB/s 
  Running setup.py bdist_wheel for clodsa ... - \ done
  Stored in directory: /root/.cache/pip/wheels/a7/e7/0b/8fc26527f6feeef05e8ed273730418ea147cf3223d65d0123a
  Running setup.py bdist_wheel for mahotas ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/fb/2a/81/12bc1070527dc81ef856f9d36db9ec0a42e8c14985d005395b
  Running setup.py bdist_wheel for commentjson ... - done
  Stored in directory: /root/.cache/pip/wheels/8c/0e/36/56d4e5ddf02e0ddd1d8f9eb8d90975a5192e0155ffe28ff19c
Successfully built clodsa mahotas commentjson


In [0]:
!pip install keras-metrics

Move to the Unet folder.

In [0]:
cd unet/

/content/unet


Load the necessary modules.

In [0]:
from matplotlib import pyplot as plt
from clodsa.augmentors.augmentorFactory import createAugmentor
from clodsa.transformers.transformerFactory import transformerGenerator
from clodsa.techniques.techniqueFactory import createTechnique
from keras.optimizers import SGD
from clodsa.utils.minivgg import MiniVGGNet
import cv2
%matplotlib inline

from model import *
from data import *
import keras_metrics

Using TensorFlow backend.


Apply the augmentation process.

In [0]:
PROBLEM = "semantic_segmentation"
ANNOTATION_MODE = "folders"
INPUT_PATH = "/content/unetData/train/"
GENERATION_MODE = "linear"
OUTPUT_MODE = "folders"
OUTPUT_PATH= "/content/unetData/train/"
PARAMETERS = {
    "batchSize": 2,
    "width": 256,
    "height": 256,
    "labelsExtension":".png",
    "outputPath": OUTPUT_PATH
  }
augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,PARAMETERS)
transformer = transformerGenerator(PROBLEM)
elastic = createTechnique("elastic",{"alpha":5,"sigma":0.05})
augmentor.addTransformer(transformer(elastic))
augmentor.applyAugmentation()

In [0]:
print("Number of nuclei images")
!ls /content/unetData/train/images/ | wc -l
print("Number of masks for nuclei images")
!ls /content/unetData/train/labels/ | wc -l

Number of nuclei images
40
Number of masks for nuclei images
40


In [0]:
ls /content/unetData/train/images/

0_0_0.png    12.png       16.png       20.png  2.png       6.png
0.png        13_0_14.png  17_0_24.png  22.png  3_0_17.png  7_0_4.png
10_0_13.png  13.png       17.png       23.png  3.png       8_0_28.png
1_0_22.png   14_0_18.png  18_0_26.png  24.png  4_0_10.png  8.png
10.png       14.png       18.png       26.png  4.png       9_0_12.png
11_0_16.png  15_0_20.png  19_0_3.png   28.png  5_0_23.png
12_0_29.png  16_0_8.png   2_0_2.png    29.png  6_0_6.png


Train the model.

In [0]:
data_gen_args = dict()
myGene = trainGenerator(2,'/content/unetData/train','images','labels',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=200,epochs=5,callbacks=[model_checkpoint])

/content/unet/model.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


Found 40 images belonging to 1 classes.
Found 40 images belonging to 1 classes.
Epoch 1/5
200/200 [==============================] - 90s 451ms/step - loss: 0.4037 - true_positive: 19412829.0000 - true_negative: 1733245.0000 - false_positive: 4098075.0000 - false_negative: 970251.0000 - acc: 0.8067 - f1_score: 0.8845 - precision: 0.8257 - recall: 0.9524

Epoch 00001: loss improved from inf to 0.40371, saving model to unet_membrane.hdf5
Epoch 2/5
200/200 [==============================] - 82s 412ms/step - loss: 0.3675 - true_positive: 18929394.0000 - true_negative: 2889246.0000 - false_positive: 2942074.0000 - false_negative: 1453686.0000 - acc: 0.8323 - f1_score: 0.8960 - precision: 0.8655 - recall: 0.9287

Epoch 00002: loss improved from 0.40371 to 0.36747, saving model to unet_membrane.hdf5
Epoch 3/5
200/200 [==============================] - 83s 413ms/step - loss: 0.3167 - true_positive: 18795665.0000 - true_negative: 3732039.0000 - false_positive: 2099281.0000 - false_negative: 1587

Evaluate the model.

In [0]:
import cv2
import glob
import numpy as np
def geneTrainNpyPrueba(image_path,mask_path,flag_multi_class = False,num_class = 2,image_as_gray = True,mask_as_gray = True):
    image_name_arr = glob.glob(image_path+"*")
    mask_name_arr = glob.glob(mask_path+"*")
    image_arr = []
    mask_arr = []
    for index,item in enumerate(image_name_arr):
        img = cv2.imread(item,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img,(256,256))
        img = np.reshape(img,img.shape + (1,)) if image_as_gray else img
        mask = cv2.imread(mask_name_arr[index],cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask,(256,256))
        mask = np.reshape(mask,mask.shape + (1,)) if mask_as_gray else mask
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        image_arr.append(img)
        mask_arr.append(mask)
    image_arr = np.array(image_arr)
    mask_arr = np.array(mask_arr)
    return image_arr,mask_arr

In [0]:
image_arr, mask_arr=geneTrainNpyPrueba('/content/unetData/test/images/','/content/unetData/test/labels/')
model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.evaluate(image_arr,mask_arr)
print(results)

/content/unet/model.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


10/10 [==============================] - 5s 494ms/step
[0.28318899869918823, 491113, 104789, 30476, 28982, 0.9092742800712585, 0.9429212197899951, 0.9415708536796326, 0.9442755650408206]


In [0]:
TP=results[1]
TN=results[2]
FP=results[3]
FN=results[4]

In [0]:
TNR=TN/(TN+FP)
print(TNR)

0.7746941189516874
